In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from dataset.connection import connect_to_spotify_dataset
from dataset.connection import select, delete, add_columns, batch_update

ModuleNotFoundError: No module named 'dataset'

In [ ]:
connection = connect_to_spotify_dataset()
if not connection:
    print("Connection Error")

In [ ]:
tds = select(connection, "SELECT * FROM track_data") 
columns = ["id", "name", "popularity", "duration_ms", "explicit", "artists", "id_artists", "release_date",
"danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", 
"instrumentalness", "liveness", "valence", "tempo", "time_signature"]
    
# Create the DataFrame
df = pd.DataFrame(tds, columns=columns)
    
# Remove tracks with durations longer than 20 min - ~1000 tracks removed
df2 = df[df["duration_ms"] <= 20 * 60 * 1000]
    
#Convert the artist column to a type that allows for similarity checks in dedup process
df2['artists'] = df2['artists'].apply(tuple)
df_deduplicated = df2.drop_duplicates(subset=['name', 'artists'], keep='first')
    
# Left Anti-join: Get rows in df that are NOT in df_deduplicated i.e. the dupes and outliers
df_duplicates = df.merge(df_deduplicated, on=["name", "artists"], how="left", indicator=True)

df_to_delete = df_duplicates[df_duplicates["_merge"] == "left_only"][["id"]]
    
delete_statements = [f"DELETE FROM track_data WHERE id = '{row['id']}';" for _, row in df_to_delete.iterrows()]
    
# Loop through each delete statement and execute it
for stmt in delete_statements:
    delete(connection, stmt)
    break

In [ ]:
    # Decade Binning
df_deduplicated["release_date"] = pd.to_datetime(df_deduplicated["release_date"], errors="coerce")
df_deduplicated["release_year"] = df_deduplicated["release_date"].dt.year
df_deduplicated["decade"] = (df_deduplicated["release_year"] // 10) * 10
    
# Combined feature index creation
df_deduplicated["mood_index"] = 0.5 * df_deduplicated["valence"] + 0.3 * df_deduplicated["danceability"] + 0.2 * df_deduplicated["energy"]
df_deduplicated["emotion_index"] = 0.4 * df_deduplicated["valence"] + 0.3 * df_deduplicated["energy"] + 0.3 * df_deduplicated["loudness"].abs()
df_deduplicated["party_index"] = 0.5 * df_deduplicated["danceability"] + 0.5 * df_deduplicated["energy"]
df_deduplicated["chill_index"] = 0.6 * df_deduplicated["acousticness"] + 0.4 * (1 - df_deduplicated["energy"])

# Add new columns to the PostgreSQL table
add_columns_query = """
ALTER TABLE your_table 
ADD COLUMN IF NOT EXISTS decade INT,
ADD COLUMN IF NOT EXISTS mood_index FLOAT,
ADD COLUMN IF NOT EXISTS emotion_index FLOAT,
ADD COLUMN IF NOT EXISTS party_index FLOAT,
ADD COLUMN IF NOT EXISTS chill_index FLOAT;
"""

# Prepare batch update query
update_query = """
    UPDATE your_table
    SET 
        decade = %s,
        mood_index = %s,
        emotion_index = %s,
        party_index = %s,
        chill_index = %s
    WHERE id = %s;
"""

# Create list of tuples for batch update
update_values = df_deduplicated[["decade", "mood_index", "emotion_index", "party_index", "chill_index", "id"]].values.tolist()
batch_update(connection,update_query,update_values)
    